IMPORTING LIBRARIES

In [45]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from operator import truediv

import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
# !pip install spectral
import spectral
import mat73
import tensorflow 
from tensorflow import keras


LOADING DATA

In [46]:
 
data_dict = {
        'data':('data (1).mat','data (2).mat','data (3).mat','data (4).mat')
    }
p=(X_1, X_2, X_3, X_4) = data_dict.get('data')
# l_rgb,l_rad=[],[]
# for i in p:
#     l_rgb.append(mat73.loadmat(i)['rgb'])
#     l_rad.append(mat73.loadmat(i)['rad'])
x=mat73.loadmat(X_1)['rgb']
y=mat73.loadmat(X_1)['rad']

TEST DATA

In [47]:
test=mat73.loadmat(X_4)['rgb']
test=np.resize(test,(64,64,3))


In [5]:

test = np.expand_dims(test, axis=0) 
test = np.expand_dims(test, axis=0) 
test.shape

(1, 1, 64, 64, 3)

In [48]:
test_y=mat73.loadmat(X_4)['rad']
test_y=np.resize(test_y,(64,64,31))
test_y=np.expand_dims(test_y,axis=0)
test_y=np.expand_dims(test_y,axis=0)
test_y

array([[[[[ 190.875     ,  191.44444444,  190.75      , ...,
            190.25      ,  188.375     ,  189.25      ],
          [ 190.5       ,  190.55555556,  190.875     , ...,
            189.875     ,  189.625     ,  190.25      ],
          [ 210.        ,  226.22222222,  244.375     , ...,
            346.125     ,  378.625     ,  577.5       ],
          ...,
          [ 209.75      ,  223.        ,  234.75      , ...,
            502.75      ,  535.625     ,  781.25      ],
          [ 211.125     ,  223.11111111,  238.875     , ...,
            452.125     ,  490.625     ,  716.375     ],
          [ 211.125     ,  222.77777778,  240.125     , ...,
            384.75      ,  417.75      ,  617.375     ]],

         [[ 211.        ,  222.66666667,  240.75      , ...,
            384.125     ,  414.5       ,  628.5       ],
          [ 209.5       ,  225.66666667,  241.125     , ...,
            409.        ,  434.625     ,  664.25      ],
          [ 211.        ,  224.55555556

TRAIN DATA

In [6]:
print(x.shape, y.shape)
train=x
valid=y

(1392, 1306, 3) (1300, 1392, 31)


In [7]:
t=np.resize(x,(64,64,3))
v=np.resize(y,(64,64,31))
# t=np.reshape(t,(1,t.shape[0],t.shape[1],t.shape[2]))
# v=np.reshape(v,(1,v.shape[0],v.shape[1],v.shape[2]))

print(t.shape,v.shape)


(64, 64, 3) (64, 64, 31)


INCRESASING DIMENSIONS TO MATCH WITH NETWORK

In [8]:
t = np.expand_dims(t, axis=0) 
v= np.expand_dims(v,axis=0)
t = np.expand_dims(t, axis=0) 
v= np.expand_dims(v,axis=0)
(t.shape,v.shape)


((1, 1, 64, 64, 3), (1, 1, 64, 64, 31))

BP-NET

In [21]:
from tensorflow import keras
from keras.layers import Conv3D
from keras.layers import Dense

#making the BP-net

cnn = keras.Sequential()
cnn.add(Conv3D(32,(3,5,5), activation='relu', input_shape=(1,64,64,3),padding='same',use_bias=True))
cnn.add(Conv3D(32, 3, activation='relu', input_shape=(32,64,64,3), padding='same', use_bias=True))
cnn.add(Conv3D(31,3, activation='relu', input_shape=(32,64,64,3), padding='same', use_bias=True))




In [22]:
# from tensorflow.keras.optimizers import SGD
# opt = SGD(lr=0.001, momentum=0.9)

cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
h = cnn.fit(x=t, y=v, epochs=310)


Epoch 1/310
1/1 [==============================] - 0s 394ms/step - loss: 106792.4766 - accuracy: 0.1294
Epoch 2/310
1/1 [==============================] - 0s 41ms/step - loss: 80746.1328 - accuracy: 0.0447
Epoch 3/310
1/1 [==============================] - 0s 36ms/step - loss: 65683.6328 - accuracy: 0.0854
Epoch 4/310
1/1 [==============================] - 0s 39ms/step - loss: 56920.9961 - accuracy: 0.1262
Epoch 5/310
1/1 [==============================] - 0s 42ms/step - loss: 52519.2930 - accuracy: 0.2217
Epoch 6/310
1/1 [==============================] - 0s 38ms/step - loss: 48032.2812 - accuracy: 0.3057
Epoch 7/310
1/1 [==============================] - 0s 41ms/step - loss: 44634.0547 - accuracy: 0.3379
Epoch 8/310
1/1 [==============================] - 0s 38ms/step - loss: 42739.9492 - accuracy: 0.3535
Epoch 9/310
1/1 [==============================] - 0s 41ms/step - loss: 41593.1562 - accuracy: 0.3052
Epoch 10/310
1/1 [==============================] - 0s 36ms/step - loss: 39865.7

In [18]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 1, 64, 64, 32)     7232      
                                                                 
 conv3d_1 (Conv3D)           (None, 1, 64, 64, 32)     27680     
                                                                 
 conv3d_2 (Conv3D)           (None, 1, 64, 64, 31)     26815     
                                                                 
Total params: 61,727
Trainable params: 61,727
Non-trainable params: 0
_________________________________________________________________


PREDICTING HSI.mat

In [39]:
y_pred=cnn.predict(test)
y_pred

array([[[[[0.10742501, 0.12843424, 0.14042543, ..., 0.12420771,
           0.1797657 , 0.18268986],
          [0.17955106, 0.18829109, 0.21023706, ..., 0.26312104,
           0.3143071 , 0.2935715 ],
          [0.22086838, 0.20294634, 0.25418198, ..., 0.33347008,
           0.37949476, 0.34533176],
          ...,
          [0.23838033, 0.21231508, 0.27526075, ..., 0.36251655,
           0.39667955, 0.36065117],
          [0.18450667, 0.17041768, 0.23597562, ..., 0.3073706 ,
           0.33007297, 0.2886588 ],
          [0.11865286, 0.08438158, 0.14235184, ..., 0.23049203,
           0.19399527, 0.1687754 ]],

         [[0.17737672, 0.19163033, 0.2005928 , ..., 0.26268458,
           0.2766465 , 0.31634232],
          [0.29830328, 0.31381756, 0.3306379 , ..., 0.49654916,
           0.49794877, 0.5221173 ],
          [0.3659795 , 0.36787578, 0.40059802, ..., 0.6156418 ,
           0.6036498 , 0.61926055],
          ...,
          [0.39446682, 0.397726  , 0.43276823, ..., 0.6627832 ,
    